## Revised Linear Layer Architecture without VIB

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import transforms
from tensorboardX import SummaryWriter
#from utils import cuda
import pdb
import time
from numbers import Number
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import argparse
from torch.utils.data import Dataset, DataLoader

### neural networks architecture

In [2]:
class ToyNet(nn.Module):
    '''
    Construct a MLP that is used to train the model 
    param[in]: X_train, output_features
    param[out]: output 
    
    Note: initialize the weight with a self-defined method
    '''

    def __init__(self, output_features=8):
        super(ToyNet, self).__init__()
        self.encode = nn.Sequential(
            nn.Linear(40, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 1024),
            nn.ReLU(True),
            nn.Linear(1024, output_features))
        #self.optim = optim.Adam(self.toynet.parameters(),lr=self.lr,betas=(0.5,0.999))
        #self.scheduler = lr_scheduler.ExponentialLR(self.optim,gamma=0.97
    def forward(self, X_train):
        output=self.encode(X_train)
        #prediction = F.softmax(output,dim=1).max(1)[1]
        
        #print(prediction)
        return output
    def weight_init(self):
        for m in self._modules:
            xavier_init(self._modules[m])
            
def xavier_init(ms):
    """
    Xavier initialization
    """
    for m in ms :
        if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
            nn.init.xavier_uniform(m.weight,gain=nn.init.calculate_gain('relu'))
            m.bias.data.zero_()

In [3]:
def cuda(tensor, is_cuda):
    if is_cuda : return tensor.cuda()
    else : return tensor

In [5]:
params={
    'epoch':50,
    'batch_size':64,
    'lr':0.01
}

In [7]:
class CustomDataset(Dataset):
    """
    construct dataset from numpy and split it 
    
    """
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)
    
    
X = joblib.load('./joblib_features/X.joblib')
y = joblib.load('./joblib_features/y.joblib')
full_dataset = CustomDataset(X, y)
train_size = int(0.6 * len(full_dataset))
valid_size = int(0.2 * len(full_dataset))
test_size = len(full_dataset) - valid_size-train_size
batch_size = params['batch_size']
#train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, 
                                                            #[train_size, test_size])
### split dataset into training, validation and test                     
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(
    full_dataset, (train_size, valid_size, test_size), generator=torch.Generator().manual_seed(42))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
my_dataloader = {'train': train_dataloader , 'validate':valid_dataloader, 'test': test_dataloader}

In [9]:
class Solver(object):
#train the model

    def __init__(self, params):
        """
        initialization of a Solver object
        
        """
        self.cuda =torch.cuda.is_available ()
        self.epoch = params['epoch']
        self.batch_size = params['batch_size']
        self.lr = params['lr']
        self.toynet = cuda(ToyNet(), self.cuda)
        self.toynet.weight_init()
        self.optim = optim.Adam(self.toynet.parameters(),
                                lr=self.lr,
                                betas=(0.5,0.999))
        self.criterion = nn.CrossEntropyLoss()
    
    def train(self):
        for epc in range(self.epoch):  # loop over the dataset multiple times
            self.toynet.train('True')  # training neural networks mode
            running_loss = 0.0
            for i, data in enumerate(train_dataloader):
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = data

                # zero the parameter gradients
                self.optim.zero_grad()

                # forward + backward + optimize
                outputs = self.toynet.forward(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optim.step()
                
                prediction = F.softmax(outputs,dim=1).max(1)[1]
                accuracy = torch.eq(prediction,labels).float().mean()
                avg_accuracy = Variable(cuda(torch.zeros(accuracy.size()), self.cuda))
                

                # print statistics
                running_loss += loss.item()
                if i % self.batch_size == 0:    # print every 30 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                            (epc + 1, i + 1, running_loss / self.batch_size))
                    print('acc:{:.4f} '
                            .format(accuracy.item(), end=' '))
                    print('err:{:.4f} '
                            .format(1-accuracy.item()))
                    running_loss = 0.0
                    
            ## validation set at each epoch
            self.test()
        print('Finished Training',(epc+1))
    
    def test(self):
        """
        Testing over a dataset
        """
        self.toynet.train('False')  # evaluation mode      
        loss, correct, total_num = 0,0,0
        
        y_real=torch.randn([0])
        y_hat=torch.randn([0])
        
        for i, data in enumerate(valid_dataloader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            outputs = self.toynet.forward(inputs)
            #loss
            total_num += labels.shape[0]
            loss += self.criterion(outputs,labels)
            prediction = F.softmax(outputs,dim=1).max(1)[1]
            y_real=torch.cat([y_real,labels],dim=0)
            y_hat=torch.cat([y_hat,prediction],dim=0)
            correct += torch.eq(prediction,labels).float().sum()
            avg_correct = Variable(cuda(torch.zeros(correct.size()), self.cuda))
        accuracy = correct/total_num
        avg_accuracy = avg_correct/total_num
        
        print('[TEST RESULT]')
        print('acc:{:.4f} '
                .format(accuracy.item(),end=' '))
        print('err:{:.4f}'
                .format(1-accuracy.item()))
        print(classification_report(y_real,y_hat))
   



In [10]:
net=Solver(params)
net.train()

C:\Users\shiji\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:36: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


[1,     1] loss: 0.663
acc:0.1094 
err:0.8906 
[TEST RESULT]
acc:0.5962 
err:0.4038
              precision    recall  f1-score   support

         0.0       1.00      0.76      0.86       115
         1.0       0.50      0.05      0.09        63
         2.0       0.83      0.45      0.58       149
         3.0       0.33      0.98      0.50       159
         4.0       0.65      0.66      0.65       180
         5.0       0.79      0.57      0.66       157
         6.0       0.97      0.28      0.44       109
         7.0       0.94      0.63      0.75       118

    accuracy                           0.60      1050
   macro avg       0.75      0.55      0.57      1050
weighted avg       0.74      0.60      0.60      1050

[2,     1] loss: 0.017
acc:0.6094 
err:0.3906 
[TEST RESULT]
acc:0.6162 
err:0.3838
              precision    recall  f1-score   support

         0.0       1.00      0.74      0.85       115
         1.0       0.31      0.71      0.43        63
         2.0      

[TEST RESULT]
acc:0.6800 
err:0.3200
              precision    recall  f1-score   support

         0.0       0.71      0.88      0.78       115
         1.0       0.00      0.00      0.00        63
         2.0       0.37      0.95      0.53       149
         3.0       0.89      0.63      0.74       159
         4.0       0.96      0.69      0.80       180
         5.0       0.84      0.61      0.70       157
         6.0       0.89      0.68      0.77       109
         7.0       0.91      0.66      0.76       118

    accuracy                           0.68      1050
   macro avg       0.70      0.64      0.64      1050
weighted avg       0.75      0.68      0.68      1050

[14,     1] loss: 0.017
acc:0.7188 
err:0.2812 


C:\Users\shiji\Anaconda3\envs\torch\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[TEST RESULT]
acc:0.7000 
err:0.3000
              precision    recall  f1-score   support

         0.0       0.72      0.88      0.79       115
         1.0       0.52      0.44      0.48        63
         2.0       0.79      0.70      0.74       149
         3.0       0.94      0.60      0.74       159
         4.0       0.47      0.97      0.64       180
         5.0       0.95      0.51      0.66       157
         6.0       0.90      0.64      0.75       109
         7.0       0.90      0.69      0.78       118

    accuracy                           0.70      1050
   macro avg       0.77      0.68      0.70      1050
weighted avg       0.78      0.70      0.71      1050

[15,     1] loss: 0.010
acc:0.7500 
err:0.2500 
[TEST RESULT]
acc:0.6914 
err:0.3086
              precision    recall  f1-score   support

         0.0       1.00      0.77      0.87       115
         1.0       0.27      0.98      0.43        63
         2.0       0.71      0.62      0.66       149
         3

[TEST RESULT]
acc:0.6305 
err:0.3695
              precision    recall  f1-score   support

         0.0       0.80      0.83      0.81       115
         1.0       0.73      0.13      0.22        63
         2.0       0.77      0.66      0.71       149
         3.0       0.54      0.72      0.61       159
         4.0       0.93      0.48      0.63       180
         5.0       0.40      0.90      0.56       157
         6.0       1.00      0.40      0.58       109
         7.0       0.82      0.63      0.71       118

    accuracy                           0.63      1050
   macro avg       0.75      0.59      0.60      1050
weighted avg       0.74      0.63      0.63      1050

[27,     1] loss: 0.015
acc:0.6719 
err:0.3281 
[TEST RESULT]
acc:0.7533 
err:0.2467
              precision    recall  f1-score   support

         0.0       0.84      0.88      0.86       115
         1.0       0.46      0.90      0.61        63
         2.0       0.80      0.69      0.74       149
         3

[TEST RESULT]
acc:0.7571 
err:0.2429
              precision    recall  f1-score   support

         0.0       1.00      0.82      0.90       115
         1.0       0.59      0.51      0.55        63
         2.0       0.67      0.81      0.73       149
         3.0       0.88      0.69      0.77       159
         4.0       0.86      0.76      0.80       180
         5.0       0.64      0.84      0.73       157
         6.0       0.68      0.76      0.72       109
         7.0       0.80      0.75      0.77       118

    accuracy                           0.76      1050
   macro avg       0.76      0.74      0.75      1050
weighted avg       0.78      0.76      0.76      1050

[39,     1] loss: 0.012
acc:0.7969 
err:0.2031 
[TEST RESULT]
acc:0.6410 
err:0.3590
              precision    recall  f1-score   support

         0.0       0.75      0.88      0.81       115
         1.0       0.52      0.51      0.51        63
         2.0       0.73      0.60      0.66       149
         3

[TEST RESULT]
acc:0.7629 
err:0.2371
              precision    recall  f1-score   support

         0.0       0.90      0.85      0.88       115
         1.0       0.58      0.67      0.62        63
         2.0       0.71      0.72      0.72       149
         3.0       0.76      0.80      0.78       159
         4.0       0.89      0.80      0.84       180
         5.0       0.87      0.68      0.76       157
         6.0       0.62      0.81      0.70       109
         7.0       0.72      0.75      0.73       118

    accuracy                           0.76      1050
   macro avg       0.75      0.76      0.75      1050
weighted avg       0.78      0.76      0.77      1050

Finished Training 50


In [8]:
 net.test()

[TEST RESULT]
acc:0.1408 
err:0.8592
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       126
         1.0       0.00      0.00      0.00        74
         2.0       0.00      0.00      0.00       151
         3.0       0.00      0.00      0.00       158
         4.0       0.00      0.00      0.00       174
         5.0       0.14      1.00      0.25       148
         6.0       0.00      0.00      0.00       104
         7.0       0.00      0.00      0.00       116

    accuracy                           0.14      1051
   macro avg       0.02      0.12      0.03      1051
weighted avg       0.02      0.14      0.03      1051



C:\Users\shiji\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:32: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [12]:
import json
dic_res = {'base_train':[],
          'base_valid':[],
          'bib_valid':[]}

In [14]:
with open('res.json', 'w') as jf:
    json.dump(dic_res, jf)

In [15]:
vib_acc = [0.23, .30, .4]